In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [95]:
class SimpleMultiClassBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50):
        self.base_estimator = base_estimator if base_estimator is not None else DecisionTreeClassifier(max_depth=1)
        self.n_estimators = n_estimators
        self.learners = []
        self.learner_weights = []
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        # Convert labels to [0, n_classes-1]
        Y = self.label_encoder.fit_transform(y)
        n_classes = len(np.unique(Y))
        # Initialize weights uniformly
        n_samples = X.shape[0]
        self.weights = (1/n_samples) * np.ones(n_samples, dtype=float)
        for _ in range(self.n_estimators):
            learner = DecisionTreeClassifier(max_depth=1)
            learner.fit(X, Y, sample_weight=self.weights)
            pred = learner.predict(X)
            missClassified = pred != y
            learner_error = np.sum(self.weights[missClassified]) / np.sum(self.weights)
            print(learner_error)
            # Compute weighted error rate (misclassification rate)
            learner_weight = np.log((1-learner_error) / learner_error) + np.log(n_classes - 1)
            # Compute learner weight using SAMME algorithm

            if learner_error >= 1 - (1 / n_classes):
                break

            # Increase the weights of misclassified samples
            for idx in range(n_samples):
                if missClassified[idx] == True:
                    self.weights[idx] *= np.exp(learner_weight)    
            self.weights /= np.sum(self.weights)    
            # Save the current learner
            self.learners.append(learner)
            self.learner_weights.append(learner_weight)

    def predict(self, X):
        # Collect predictions from each learner

        # Weighted vote for each sample's prediction across all learners

        # Final prediction is the one with the highest weighted vote

        # Convert back to original class labels
        predictionsOfLearners = []
        for learner in self.learners:
            predictionsOfLearners.append(learner.predict(X))
        predictionsOfLearners = np.array(predictionsOfLearners)
        prediction = np.empty(X.shape[0])        
        for i in range(X.shape[0]):
            labels = np.unique(predictionsOfLearners[:, i])
            votes = {label : 0 for label in labels}
            for j in range(len(predictionsOfLearners[:, i])):
                for label in labels:
                    if predictionsOfLearners[j, i] == label:
                        votes[label] += self.learner_weights[j]
            finalPrediction = max(votes, key=votes.get)
            prediction[i] = self.label_encoder.inverse_transform(np.array([finalPrediction]))
        return prediction


In [18]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, :-1].to_numpy(), df.iloc[:, -1].to_numpy(), test_size=0.2, random_state=42)

In [96]:
m = SimpleMultiClassBoosting()
m.fit(X_train, Y_train)

0.32500000000000023
0.18581829692940816
0.12491488735987881
0.2634006663297185
0.1705713068098068
0.15338144166126846
0.3018000494584266
0.19759964963833307
0.23989307969899254
0.2923565307552004
0.26043915582752286
0.14724732677712646
0.25947684533181675
0.27252734894196573
0.30547158288221943
0.29430175351792454
0.24880935630761938
0.25210016520492967
0.35809837351013407
0.22094842913424811
0.35654992076021946
0.227299256601732
0.25298390542317023
0.3094757042107903
0.2751459933081308
0.2230733666498616
0.320670908376389
0.2887891370992855
0.31245616991264175
0.2894173611823568
0.22975093898891885
0.288506969330827
0.2401463884170762
0.29462695638674635
0.29438210282190935
0.26700683818104365
0.2549891583734744
0.35600973710893846
0.2821296276884286
0.2696812360491574
0.27022560332429146
0.30899296715094055
0.26338608371369104
0.26256484333792873
0.31549983822241356
0.2335035059775038
0.259481750651754
0.30009013608754037
0.32002090584134024
0.2927479649300975


In [97]:
prediction = m.predict(X_test)
prediction

array([1., 0., 2., 1., 1., 0., 1., 2., 2., 1., 1., 0., 0., 0., 0., 1., 2.,
       1., 1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0.])

In [98]:
print(m.learner_weights)

[1.4240346891027367, 2.170561446089358, 2.639835729512814, 1.721514960937093, 2.2747308918393885, 2.401469527859688, 1.5318879941819428, 2.094511841622896, 1.8464129657291326, 1.577113519708257, 1.7368344697422455, 2.4495030686278154, 1.7418366121269213, 1.6749846136945377, 1.5145235033971005, 1.5677293002828503, 1.7981196873318779, 1.7805897493570746, 1.2767745077474575, 1.9532951036544404, 1.283517367413939, 1.9167715572845565, 1.775908039303081, 1.4957187496653785, 1.66181560350281, 1.9409923877147932, 1.44383747202999, 1.5944193462987593, 1.481808557862806, 1.591362629463538, 1.9028652507352972, 1.595793559994896, 1.8450242898142992, 1.566163981809293, 1.5673424564506098, 1.7030092841330329, 1.7653249232803097, 1.2858727049080287, 1.627069555377835, 1.6893876280319098, 1.6866254433145942, 1.4979786654508649, 1.7215901223564782, 1.7258272679719417, 1.4676778924171834, 1.881780212963689, 1.7418110835021503, 1.5400158583187902, 1.4468229099591712, 1.5752222172869053]


In [99]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, prediction))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.89      0.89      0.89         9
         2.0       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

